In [1]:
import pyTigerGraph as tg

conn = tg.TigerGraphConnection(graphname="Synthea", gsqlVersion="3.0.0")

In [3]:
conn.gsql('''
          DROP QUERY getTriplets 

          CREATE QUERY getTriplets(/* Parameters here */) FOR GRAPH Synthea SYNTAX v2{ 
            TYPEDEF TUPLE <VERTEX src, EDGE edg, VERTEX dest> TUPLE_RECORD;
            SetAccum<TUPLE_RECORD> @@triplets;
            p = ANY;
            res = SELECT tgt FROM p:s-(:e1)-:tgt WHERE s!= tgt
                    ACCUM @@triplets += TUPLE_RECORD(s, e1, tgt);
            PRINT @@triplets;
          }
            
          INSTALL QUERY getTriplets
          ''')

'========================\nTrying version: v3_0_0\nConnecting to localhost:14240\nWARNING: sun.reflect.Reflection.getCallerClass is not supported. This will impact performance.\nIf there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql\nQuery getTriplets could not be found.\nThe query getTriplets has been added!\nStart installing queries, about 1 minute ...\ngetTriplets query: curl -X GET \'http://127.0.0.1:9000/query/Synthea/getTriplets\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\n\n\r[                                                                 ] 0% (0/1)   \r[                                                                 ] 0% (0/1)   \r[=====                                                            ] 7% (0/1)   \r[==========                                                       ] 14% (0/1)  \r[==============                                                   ] 21% (0/1)  \r[===================                                     

In [13]:
res = conn.runInstalledQuery("getTriplets", timeout=512000, sizeLimit=5.12e+8)

In [14]:
import pandas as pd

df = pd.DataFrame(res[0]["@@triplets"])

In [15]:
df.head()

,src,edg,dest
0,f8dc86a0-2d89-60b2-17c9-effc000b358f,"{'e_type': 'patientSeen', 'from_id': '34ce6eaf...",34ce6eaf-b4c0-87b8-0d5f-d52bd3106fc9
1,a6922a64-3a22-37b2-a8ba-f1f45017573e,"{'e_type': 'providerSeen', 'from_id': 'ac3c275...",ac3c275f-5640-025c-5a8a-54c8ed5a8c86
2,5bc2e490-0843-343f-a992-16a52b7ddc8a,"{'e_type': 'providerSeen', 'from_id': '16a0dac...",16a0dacb-a362-50ad-90dc-8238810e4690
3,ed1f3e5f-2034-386c-bc53-cc21649294ee,"{'e_type': 'providerSeen', 'from_id': '4cfbdb8...",4cfbdb87-eabe-f0b3-cd6b-5ac93b17c60b
4,4535458f-a191-fb1b-7ddd-d1ea8c63ea41,"{'e_type': 'patientVisit', 'from_id': '40d6fd0...",40d6fd08-f01d-5096-7a18-a35fed7b99c6


In [16]:
df = pd.concat([df.drop(['edg'], axis=1), df['edg'].apply(pd.Series)], axis=1)
df.head()

,src,dest,e_type,from_id,from_type,to_id,to_type,directed,attributes
0,f8dc86a0-2d89-60b2-17c9-effc000b358f,34ce6eaf-b4c0-87b8-0d5f-d52bd3106fc9,patientSeen,34ce6eaf-b4c0-87b8-0d5f-d52bd3106fc9,Visit,f8dc86a0-2d89-60b2-17c9-effc000b358f,Patient,True,{}
1,a6922a64-3a22-37b2-a8ba-f1f45017573e,ac3c275f-5640-025c-5a8a-54c8ed5a8c86,providerSeen,ac3c275f-5640-025c-5a8a-54c8ed5a8c86,Visit,a6922a64-3a22-37b2-a8ba-f1f45017573e,Provider,True,{}
2,5bc2e490-0843-343f-a992-16a52b7ddc8a,16a0dacb-a362-50ad-90dc-8238810e4690,providerSeen,16a0dacb-a362-50ad-90dc-8238810e4690,Visit,5bc2e490-0843-343f-a992-16a52b7ddc8a,Provider,True,{}
3,ed1f3e5f-2034-386c-bc53-cc21649294ee,4cfbdb87-eabe-f0b3-cd6b-5ac93b17c60b,providerSeen,4cfbdb87-eabe-f0b3-cd6b-5ac93b17c60b,Visit,ed1f3e5f-2034-386c-bc53-cc21649294ee,Provider,True,{}
4,4535458f-a191-fb1b-7ddd-d1ea8c63ea41,40d6fd08-f01d-5096-7a18-a35fed7b99c6,patientVisit,40d6fd08-f01d-5096-7a18-a35fed7b99c6,Patient,4535458f-a191-fb1b-7ddd-d1ea8c63ea41,Visit,True,{}


In [17]:
df.shape

(6182004, 9)

In [18]:
entities = pd.DataFrame({"entity":df["from_id"].append(df["to_id"]), "type":df["from_type"].append(df["to_type"])})
entities = entities.drop_duplicates(["entity", "type"])

In [19]:
entities.head()

,entity,type
0,34ce6eaf-b4c0-87b8-0d5f-d52bd3106fc9,Visit
1,ac3c275f-5640-025c-5a8a-54c8ed5a8c86,Visit
2,16a0dacb-a362-50ad-90dc-8238810e4690,Visit
3,4cfbdb87-eabe-f0b3-cd6b-5ac93b17c60b,Visit
4,40d6fd08-f01d-5096-7a18-a35fed7b99c6,Patient


In [20]:
entities.shape

(475408, 2)

In [21]:
hrt = df[["src", "e_type", "dest"]]

In [22]:
hrt.to_csv("./data/train.tsv", index=False, header=False, sep="\t")

In [23]:
!dglke_train --model_name DistMult --data_path /Users/pericks4/kgEmbedding/data --dataset Synthea --format raw_udd_hrt --data_files ~/kgEmbedding/data/train.tsv --hidden_dim 216

9000/80000) average regularization: 0.0318770556524396
[proc 0][Train] 1000 steps take 36.885 seconds
[proc 0]sample: 3.397, forward: 21.049, backward: 8.645, update: 3.785
[proc 0][Train](30000/80000) average pos_loss: 0.25229762417078017
[proc 0][Train](30000/80000) average neg_loss: 0.5177760181427002
[proc 0][Train](30000/80000) average loss: 0.38503682121634486
[proc 0][Train](30000/80000) average regularization: 0.031759304972365496
[proc 0][Train] 1000 steps take 37.392 seconds
[proc 0]sample: 3.411, forward: 21.486, backward: 8.644, update: 3.842
[proc 0][Train](31000/80000) average pos_loss: 0.2528031569421291
[proc 0][Train](31000/80000) average neg_loss: 0.5138676720261574
[proc 0][Train](31000/80000) average loss: 0.38333541506528857
[proc 0][Train](31000/80000) average regularization: 0.031725066980347036
[proc 0][Train] 1000 steps take 37.311 seconds
[proc 0]sample: 3.465, forward: 21.344, backward: 8.666, update: 3.826
[proc 0][Train](32000/80000) average pos_loss: 0.253

In [ ]:
import numpy as np

In [26]:
entityEmb = np.load("ckpts/DistMult_Synthea_0/Synthea_DistMult_entity.npy")

In [46]:
entityEmb

array([[-0.00054407,  0.01988553,  0.01895951, ..., -0.01120941,
        -0.0274428 ,  0.01791391],
       [-0.04992942,  0.04578114,  0.00835377, ..., -0.01925704,
        -0.06534197, -0.04428214],
       [ 0.04662586, -0.05664321, -0.09248331, ..., -0.07491291,
         0.05608689,  0.0256738 ],
       ...,
       [ 0.05581236, -0.01671547,  0.01964582, ..., -0.05025264,
         0.01698434,  0.02239789],
       [ 0.02769739, -0.05354497,  0.0375741 , ..., -0.04679642,
         0.00346978,  0.04622282],
       [-0.04516835,  0.02760208,  0.02439669, ...,  0.02669085,
         0.05012084, -0.03910637]], dtype=float32)

In [47]:
entityEmb.shape

(468431, 216)

In [48]:
relationEmb = np.load("ckpts/DistMult_Synthea_0/Synthea_DistMult_relation.npy")

In [49]:
relationEmb

array([[-0.0568258 , -0.02092931, -0.04164599, ..., -0.04973273,
         0.04875431, -0.01233586],
       [ 0.04526114,  0.00073258, -0.06260296, ..., -0.06403837,
        -0.0416248 , -0.05394542],
       [-0.05705387, -0.022148  , -0.04163705, ..., -0.04990689,
         0.04869061, -0.01309733],
       ...,
       [ 0.35084787,  0.3299731 ,  0.28466958, ...,  0.3317943 ,
         0.30453217,  0.348263  ],
       [-0.05808233,  0.02506154,  0.04322441, ..., -0.01765681,
        -0.03564592,  0.03560021],
       [ 0.04393549, -0.02285082,  0.07482006, ..., -0.09750175,
        -0.02211131,  0.05283095]], dtype=float32)

In [50]:
relationEmb.shape

(38, 216)

In [51]:
import pandas as pd
entIdx = pd.read_csv("data/entities.tsv", sep="\t", header=None, index=0)
entIdx.rename(columns={1:"entity", 0:"idx"}, inplace=True)
entities = pd.merge(entities, entIdx, on="entity")

In [52]:
entities

,entity,type,idx_x,embedding,pca0,pca1,pca2,idx_y
0,f8dc86a0-2d89-60b2-17c9-effc000b358f,Patient,0,"[-0.0005440739, 0.019885529, 0.01895951, -0.00...",0.860326,1.236253,0.105194,0
1,34ce6eaf-b4c0-87b8-0d5f-d52bd3106fc9,Visit,1,"[-0.049929418, 0.04578114, 0.0083537735, -0.02...",0.182867,0.008009,-0.026596,1
2,a6922a64-3a22-37b2-a8ba-f1f45017573e,Provider,2,"[0.04662586, -0.05664321, -0.09248331, -0.0428...",-1.038904,1.305906,0.381209,2
3,ac3c275f-5640-025c-5a8a-54c8ed5a8c86,Visit,3,"[-0.029613135, 0.019209772, -0.007834088, 0.04...",0.209875,-0.038801,-0.061442,3
4,5bc2e490-0843-343f-a992-16a52b7ddc8a,Provider,4,"[-0.029931884, -0.03688564, 0.036687057, 0.019...",-0.772458,1.422599,0.341634,4
...,...,...,...,...,...,...,...,...
475403,2bd8a500-d99a-35b5-fbb7-19f9d5d1e138,CarePlan,468426,"[0.017153049, 0.0152853485, 0.061778165, -0.01...",-0.795419,0.037683,0.190206,468426
475404,8f157b1f-fe4f-5196-48dd-bc0f5ab5ec60,CarePlan,468427,"[-0.01690485, -0.0016199874, -0.05795732, -0.0...",0.101799,-0.079915,-0.029866,468427
475405,79f3d370-76a7-b331-2717-a693232aa317,CarePlan,468428,"[0.05581236, -0.016715467, 0.019645821, 0.0109...",0.095591,-0.064162,-0.036994,468428
475406,fb15c3d6-1a6d-83ad-ee2b-71f9c6ddb85b,CarePlan,468429,"[0.02769739, -0.05354497, 0.0375741, -0.039909...",0.159966,-0.027664,0.044850,468429


In [53]:
patIdxs = entities[entities["type"] == "Patient"]["idx"]

KeyError: 'idx'

In [ ]:
pats = entityEmb[patIdxs]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sims = cosine_similarity(pats)

In [ ]:
sims

In [ ]:
sims.shape

In [ ]:
entities = entities.sort_values("idx").reset_index().drop("index", axis=1)

In [ ]:
entities["embedding"] = [entityEmb[i] for i in entities["idx"]]

In [ ]:
entities

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pcaRes = pca.fit_transform(np.stack(entities["embedding"].values))

In [ ]:
entities["pca0"] = [pcaRes[i][0] for i in entities["idx"]]
entities["pca1"] = [pcaRes[i][1] for i in entities["idx"]]
entities["pca2"] = [pcaRes[i][2] for i in entities["idx"]]

In [ ]:
entities.head()

In [ ]:
entities.to_csv("entitiesWithEmbeddingsDistMult.csv", sep="\t")